CAP5610 HW4 - Yuan Du

Code stored at my Github: https://github.com/YuanEldaif/CAP5610


## Task 1: 
### Q1: What is the margin and support vectors? 
Margin is the distance/width from the decision boudary to the closest data point.
Support vectors are the points when the margin is maximized in which the hyperplane that has the largest perpendicular distance between the hyperplane and the closest samples on either side.

### Q2: How does SVM deal with non-separable data? 
By using kernal tricks/functions.

### Q3: What is a kernel? 
A kernel is a mathematical function that maps data points to higher dimensional space to linearly seperate non-linearly separable data. 

### Q4: How does a kernel relate to feature vectors?
A kernel specifies the inner product under some feature mapping to a feature vector in high dimentional feature space.

## Task 2: 
### Construct a support vector machine that computes the kernel function. Use four values of +1 and -1for both inputs and outputs: 
### Map the input [x1, x2] into a space consisting of x1 and x1x2. Draw the four input points in this space, and the maximal margin separator. What is the margin?

The examples $[x_1,x_2]$ maps to $[x_1, x_1x_2]$ as below:

$[-1,-1]$ (negative) maps to $[-1,1]$

$[-1,1]$ (positive) maps to $[-1,-1]$

$[1,-1]$ (positive) maps to $[1,-1]$

$[1,1]$ (negative) maps to $[1,1]$

Thus, the positive examples have $x_1x_2$ = −1 and the negative examples have $x_1x_2$ = +1.The drawings is shown in a seperate pdf file "CAP5610 HW4 Task 2". The maximimal margin separator is the line $x_1x_2$ = 0, with a margin of 1. 

## Task 3: 
### Recall that the equation of the circle in the 2-dimensional plane is $(x1 - a)^2 + (x2 - b)^2 - r^2 = 0$. Please expand out the formula and show that every circular region is linearly separable from the rest of the plane in the feature space (x1, x2, x12, x22).
The circle equation expands into below terms: 

$x_1^2 + x_2^2 - 2ax_1 - 2bx_2 + (a^2 +b^2 -r^2) = 0$

Thus, weights in the feature space :w = (2a, 2b, 1, 1) and intercept $(a^2 +b^2 -r^2)$. This shows that a circular boundary is linear in this feature space, allowing linear separability.

## Task 4: 
### Recall that the equation of an ellipse in the 2-dimensional plane is $c(x1 - a)^2 + d(x2 - b)^2 - 1 = 0$. Please show that an SVM using the polynomial kernel of degree 2, $K(u, v) = (1 + u . v)^2$, is equivalent to a linear SVM in the feature space (1, x1, x2, x12, x22, x1x2) and hence that SVMs with this kernel can separate any elliptic region from the rest of the plane.
The ellipse equation expands into below terms:
$cx_1^2 + dx_2^2 - 2acx_1 - 2bdx_2 + (ca^2 +db^2 -1) = 0$

Thus, weights in the feature space :w = (1, 2a, 2b, c, d, 0) and intercept $(ca^2 +db^2 -1)$. This shows that an elliptical boundary is linear in this feature space, allowing linear separability.

## Task 5: 
### (a) Plot these six training points. Are the classes {+, −} linearly separable?
Yes. The classes are linearly separable

### (b) Construct the weight vector of the maximum margin hyperplane by inspection and identify the support vectors.
The maximum margin hyperplane should have a slope of −1 and should satisfy x1 = 3/2, x2 = 0. Thus, the equation is $x_1 +x_2 = 3/2$ and the weight vector is $(1,1)^T$

## Task 6: 
### Consider a dataset with 3 points in 1-D:
### (a) Are the classes {+, -} linearly separable?
No. The classes are not linearly separable.

### (b) Consider mapping each point to 3-D using new feature vectors ?x) = [1, sqrt(2)x, x^2 ] . Are the classes now linearly separable? If so, find a separating hyperplane.
Yes. They are now linearly separable. The points are mapped to $(1, 0, 0),(1, \sqrt{2},1),(1,-\sqrt{2},1)$ respectively. A separating hyperplane is given by the weight vector (0,0,1) in the new space.

## Task 7:
### Learning SVMs on the Titanic dataset. Please report your five-fold cross validation classification accuracies on Titanic training set, with respect to the linear, quadratic, and RBF kernels. Which kernel is the best in your case?

With default SVM setting, linear kernel is the best. After parameter tuning, rbf kernel is the best as shown in below table: 

In [1]:
#Data analysis and wrangling
import pandas as pd
import numpy as np

#Data Visualization
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Load in the train and test datasets
train = pd.read_csv('C:/Work/Project/CAP5610/HW/HW1/train.csv')
test = pd.read_csv('C:/Work/Project/CAP5610/HW/HW1/test.csv')

combine = [train, test]

# Store our passenger ID for easy access
PassengerId = test['PassengerId']

#Q14 drop ticket
#Q15 drop cabin
train = train.drop(['Ticket', 'Cabin'], axis=1)
test = test.drop(['Ticket', 'Cabin'], axis=1)
combine = [train, test]

#Q16 Convert sex to numeric

for dataset in combine:
    dataset['Sex'] = dataset['Sex'].map( {'female': 1, 'male': 0} ).astype(int)

#Q17 
# fill missing values with median column values
train.fillna(train.median(), inplace=True)
test.fillna(test.median(), inplace=True)

#Q18 Fill embarked with mode
freq_port = train.Embarked.dropna().mode()[0]
for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].fillna(freq_port)
    
#Q19 complete Fare using mode
test['Fare'].fillna(test['Fare'].dropna().median(), inplace=True)    

# update combine
combine = [train, test]

#Q20 Convert the Fare feature to ordinal values
for dataset in combine:
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
    dataset.loc[ dataset['Fare'] > 31, 'Fare'] = 3
    dataset['Fare'] = dataset['Fare'].astype(int)
    
#Convert categorical variables into numeric
for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)

#Drop Name,PassengerId
train = train.drop(['Name','PassengerId'], axis=1)
test = test.drop(['Name','PassengerId'], axis=1)

X=train.drop(columns=['Survived','SibSp','Parch','Embarked']).values
y=train['Survived'].values

In [2]:
from sklearn.preprocessing import StandardScaler

#Scale data
scaler = StandardScaler()  
# Don't cheat - fit only on training data
scaler.fit(X)
#scaler.fit(test)
train = scaler.transform(X)
#test = scaler.transform(test)

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
X_train.shape, y_train.shape, X_test.shape, y_test.shape



((712, 4), (712,), (179, 4), (179,))

In [4]:
from sklearn.svm import SVC

#Model selection and evaluation
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score
from sklearn import metrics

In [5]:
#linear
clf_svm_lin = SVC(kernel='linear')
#quadratic
clf_svm_qua = SVC(kernel='poly', degree=2)
#RBF
clf_svm_rbf = SVC(kernel='rbf')

In [6]:
#Tuning parameters for each SVM kernels

#a) Select Parameters for different kernels
#parameters = {'kernel':['linear','poly', 'rbf'],'C':[0.1, 1, 5, 10]}
kernel = ['linear','poly', 'rbf']
C = [50, 10, 1.0, 0.1, 0.01]
gamma = ['scale'] #default

svc_grid_lin = dict(kernel=['linear'],C=C,gamma=gamma)
svc_grid_qua = dict(kernel=['poly'],C=C,gamma=gamma)
svc_grid_rbf = dict(kernel=['rbf'],C=C,gamma=gamma)
grids = [svc_grid_lin,svc_grid_qua,svc_grid_rbf]

#b) Decide Metric: Accuracy
from sklearn.metrics import make_scorer
scorer = make_scorer(metrics.accuracy_score)

#c) Create GridSearch Object 
from sklearn.model_selection import GridSearchCV



In [7]:
#Results dataframe
cols = ['Model','SVM-linear','SVM-Poly','SVM-rbf']

resul = pd.DataFrame(columns=cols)
resul.set_index("Model",inplace=True)
resul.loc['Default'] = [0,0,0]
resul.loc['GridSearch'] = [0,0,0]


In [8]:
#default results for SVM with different kernels 
models = [clf_svm_lin,clf_svm_qua,clf_svm_rbf]
col = 0
for model in models:
    model.fit(X_train,y_train)
    resul.iloc[0,col] = model.score(X_test,y_test)
    col += 1


In [9]:
#Summary results for SVM with different kernels - default & gridsearch
col = 0
for ind in range(0,len(models)):
    grid_search = GridSearchCV(estimator=models[col], 
                  param_grid=grids[col], n_jobs=-1, cv=5,  
                  scoring=scorer,error_score=0)
    grid_clf_acc = grid_search.fit(X_train, y_train)
    resul.iloc[1,col] = grid_clf_acc.score(X_test,y_test)
    col += 1
    
resul.head()

,SVM-linear,SVM-Poly,SVM-rbf
Model,,,
Default,0.782123,0.620112,0.608939
GridSearch,0.782123,0.77095,0.793296
